In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import csv,os,re
import sklearn
import numpy as np
import pickle
import random
import gensim

C:\Users\Mudit\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
#Build word embeddings model from cluster-wise corpus. 
#Get vector representation for the n-nearest neighbouring points in the same cluster as the target point and use cosine similarity

In [102]:
#Loading the artist-level k-means clustering model:

In [2]:
artist_model = pickle.load(open('Pickled models/artist-level-clustering-model','rb'))

C:\Users\Mudit\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator KMeans from version 0.20.3 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
df = pd.read_csv(r'Data/lyrics_normalized.csv')

We will pick a song at random from our data set and use it to get the top-5 recommendations. The same approach can be applied to an external song, after extracting its required numeric features. 

In [19]:
song_ind = random.randint(0,len(df)) #Taking a song from our dataset at random

In [20]:
song = df.iloc[song_ind]

In [29]:
#Order of features to take: polarity,ttr,unique_word_count,word density
vals = np.array(list([song['Sentiment Polarity'],song['Type-Token Ratio'],song['Unique_Word_Count'],song['Word Density']]))

In [37]:
artist_mean_array = pd.concat([df['Sentiment Polarity'],df['Type-Token Ratio'],df['Unique_Word_Count'],df['Word Density']],axis = 1).values

In [24]:
vals

array([-0.12815657,  0.66326531,  0.80634921,  0.82873653])

In [31]:
prediction = artist_model.predict(vals.reshape(1,4))[0] #Determine the cluster number for this song

In [32]:
#hasattr(artist_model.labels_,'next')
prediction

2

In [67]:
from gensim.models import Word2Vec

In [77]:
#Generate four corpuses - one for each cluster
corpus_one = ''
corpus_two = ''
corpus_three = ''
corpus_four = ''


In [33]:
def get_distance(a,b):
    return sum([(x-y)**2 for x,y in zip(a,b)])

In [43]:
def get_top_n(n,song,prediction): #prediction is the label prediction for the target point i.e. the song whose most similar songs are to be found
    #song is the 1-d numpy array representing the target song
    indices = []
    for ind,val in enumerate(artist_model.labels_):
        if val == prediction:
            indices.append(ind) #These indices correspond to datapoints belonging in the same cluster as the target point
    distances = []
    for ind in indices:
        datapoint = artist_mean_array[ind,:]
        dist = get_distance(song,datapoint)
        distances.append((ind,dist))
    return sorted(distances,key = lambda x:x[1])[:n]

In [ ]:
#Use 100 as argument value for parameter n to get the 100 most similar artists.Then, build corpus

In [44]:
res = get_top_n(100,vals,prediction)

In [47]:
res

[(1769, 0.0012786268008515525),
 (2013, 0.0023981115643599395),
 (592, 0.003027597434217502),
 (2519, 0.003180029603945762),
 (850, 0.0040265272436803),
 (1426, 0.004040578467193018),
 (3095, 0.005150925919885154),
 (1411, 0.0052555797806879126),
 (815, 0.005480811271986962),
 (570, 0.0063644840222861826),
 (2849, 0.007439872697623466),
 (3443, 0.00755902780984986),
 (3986, 0.007640466068135841),
 (3151, 0.007925803081855552),
 (3682, 0.008666167060984272),
 (1477, 0.008727059450347283),
 (3260, 0.009389405966479225),
 (660, 0.009416581547111877),
 (4292, 0.009479778571307116),
 (4204, 0.009511406884512406),
 (3700, 0.00984066081650068),
 (2838, 0.010114581944765028),
 (2629, 0.010524543965403086),
 (3072, 0.011684407975948673),
 (999, 0.011691037708734107),
 (2532, 0.011769455291785597),
 (3050, 0.011879581099309918),
 (4061, 0.012060540854863244),
 (3024, 0.012256971851353002),
 (318, 0.012460800218087355),
 (1978, 0.012545133882672976),
 (3127, 0.012784769287691755),
 (3451, 0.01306

In [48]:
artist_list = pickle.load(open('Pickled models/artist-list','rb'))

In [57]:
def build_corpus():
    corpus = ''
    nearest_n_indices = [x[0] for x in res]
    for n in nearest_n_indices:
        artist = artist_list[n]
        artist_df = df[df['artist'] == artist]
        artist_lyrics = ' '.join(list(artist_df['lyrics']))
        corpus += artist_lyrics
    return corpus
    
    

In [60]:
corpus = build_corpus()

In [61]:
def get_songs():
    num_artists = len(res)
    

2953273

100

In [112]:
for ind,cluster in enumerate(artist_model.labels_):
    if cluster == 0:
        corpus_one += df.iloc[ind]['lyrics']
    elif cluster == 1:
        corpus_two += df.iloc[ind]['lyrics']
    elif cluster == 2:
        corpus_three += df.iloc[ind]['lyrics']
    else:
        corpus_four += df.iloc[ind]['lyrics']

In [90]:
#Build word embeddings for each cluster:


In [91]:
embedding_one = Word2Vec(corpus_one,size = 100,min_count = 5)

C:\Users\Mudit\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [97]:
pickle.dump(embedding_one,open('Pickled models/artist_embedding_one','wb'))

In [92]:
embedding_two = Word2Vec(corpus_two,size = 100,min_count = 5)

In [98]:
pickle.dump(embedding_two,open('Pickled models/artist_embedding_two','wb'))

In [93]:
embedding_three = Word2Vec(corpus_three,size = 100,min_count = 5)

In [99]:
pickle.dump(embedding_three,open('Pickled models/artist_embedding_three','wb'))

In [95]:
embedding_four = Word2Vec(corpus_four,size = 100,min_count = 5)

In [100]:
pickle.dump(embedding_four,open('Pickled models/artist_embedding_four','wb'))

Loading the pickled embedding models:

In [123]:
embedding_one = pickle.load(open('Pickled models/artist_embedding_one','rb'))
embedding_two = pickle.load(open('Pickled models/artist_embedding_two','rb'))
embedding_three = pickle.load(open('Pickled models/artist_embedding_three','rb'))
embedding_four = pickle.load(open('Pickled models/artist_embedding_four','rb'))
    

In [154]:
artist_mean_array = pickle.load(open('Pickled models/artist-mean-array','rb'))

In [115]:
features = [x for x in vals]

In [167]:
distances = get_top_n(5,vals,prediction)

In [185]:
top_5 = sorted(distances,key = lambda x:x[1])[:5]
top_5 = [x[0] for x in top_5]

In [181]:
artist_list = pickle.load(open('Pickled models/artist-list','rb'))



In [190]:
#top_5
print('The most similar artists to the one in the target song are:')
print(artist_list[top_5[0]])
print(artist_list[top_5[1]])
print(artist_list[top_5[2]])
print(artist_list[top_5[3]])
print(artist_list[top_5[4]])


The most similar artists to the one in the target song are:
brain-failure
flaming-lips
edith-frost
arcs
blue-man-group


In [188]:
type(artist_list)

list

In [134]:
artist_model.labels_[0:100]

array([3, 3, 1, 1, 2, 2, 3, 2, 1, 2, 2, 3, 3, 0, 0, 3, 0, 3, 3, 0, 2, 3,
       3, 2, 3, 0, 3, 2, 2, 1, 2, 2, 0, 1, 3, 3, 2, 1, 3, 2, 2, 3, 3, 0,
       3, 1, 3, 1, 1, 1, 3, 3, 2, 3, 3, 3, 0, 2, 0, 2, 1, 3, 1, 0, 2, 2,
       1, 3, 3, 2, 1, 2, 3, 3, 1, 2, 0, 3, 2, 1, 3, 1, 2, 3, 3, 3, 3, 1,
       3, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1])

In [131]:
(artist_model.labels_)

array([3, 3, 1, ..., 3, 2, 2])

In [137]:
indices = []
for ind,val in enumerate(artist_model.labels_):
    if val == 2:
        indices.append(ind)
        

In [139]:
len(indices)

1469

In [140]:
indices2 = []


4940

In [157]:
len(artist_model.labels_)

4940

In [144]:
df.columns[4:]

Index(['Unique_Word_Count', 'Type-Token Ratio', 'Sentiment Polarity',
       'Word Density'],
      dtype='object')

In [176]:
(artist_model.labels_)

array([3, 3, 1, ..., 3, 2, 2])

In [178]:
len(corpus_one) + len(corpus_two) + len(corpus_three) + len(corpus_four)

11702324

In [12]:
subset = df[df['artist'] == 'beyonce-knowles']

In [17]:
lyrics = ' '.join(list(subset['lyrics']))

In [18]:
len(lyrics)

464899